In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
import pandas as pd
from tensorflow.keras.applications import ResNet50V2

tf.__version__

'2.3.0-dev20200522'

In [2]:
PATH = 'data'

os.listdir(PATH)

['jpeg224',
 'sample_submission.csv',
 'subset.csv',
 'test.csv',
 'train.csv',
 'train_split.csv',
 'val_split.csv']

In [3]:
train = pd.read_csv(f'{PATH}/subset.csv')
val = pd.read_csv(f'{PATH}/val_split.csv')

train.shape, val.shape

((2220, 8), (10932, 8))

In [4]:
train.head()

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target
0,ISIC_0533349,IP_5208504,female,45.0,lower extremity,unknown,benign,0
1,ISIC_8814612,IP_0414408,male,50.0,torso,unknown,benign,0
2,ISIC_6515241,IP_6245507,male,45.0,lower extremity,unknown,benign,0
3,ISIC_5075261,IP_2117218,male,40.0,upper extremity,unknown,benign,0
4,ISIC_2624460,IP_1969685,male,50.0,torso,unknown,benign,0


In [5]:
BATCH_SIZE = 64
AUTO = tf.data.experimental.AUTOTUNE

def decode(filename, label):
    img = tf.io.read_file(filename)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.cast(img, tf.float32)
    return img, label

def load_ds(df):
    options = tf.data.Options()
    options.experimental_deterministic = False
    imgs, labels = df["image_name"].values, df["target"].values
    imgs = [f'{PATH}/jpeg224/train/{img}.jpg' for img in imgs]
    ds = tf.data.Dataset.from_tensor_slices((imgs, labels))
    ds = ds.with_options(options)
    ds = ds.map(decode)
    ds = ds.cache()
    ds = ds.shuffle(2048)
    ds = ds.batch(BATCH_SIZE)
    ds = ds.prefetch(buffer_size=AUTO)
    return ds

In [6]:
train_ds = load_ds(train)
val_ds = load_ds(val)

In [7]:
IMAGE_SIZE = (224, 224, 3)

encoder = ResNet50V2(
    include_top=False,
    input_shape=IMAGE_SIZE,
    weights='imagenet'
)
encoder.trainable = False

inputs = keras.Input(shape=IMAGE_SIZE)
x = keras.layers.experimental.preprocessing.Rescaling(1./255)(inputs)
x = encoder(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
outputs = keras.layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
rescaling (Rescaling)        (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50v2 (Functional)      (None, 7, 7, 2048)        23564800  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 2049      
Total params: 23,566,849
Trainable params: 2,049
Non-trainable params: 23,564,800
_________________________________________________________________


In [8]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=[keras.metrics.AUC(name="auc")]
)

In [9]:
filepath = './checkpoints/checkpoint'
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath = filepath,
    monitor="val_auc",
    verbose=0,
    save_best_only=True,
    save_weights_only=True,
    mode="max"
)

In [10]:
model.fit(train_ds, 
          epochs=10, 
          validation_data=val_ds, 
          validation_steps=10,
          callbacks=[model_checkpoint])

Epoch 1/10
35/35 [==============================] - 16s 444ms/step - loss: 0.1805 - auc: 0.4019 - val_loss: 0.1355 - val_auc: 0.4009
Epoch 2/10
35/35 [==============================] - 14s 402ms/step - loss: 0.1080 - auc: 0.4507 - val_loss: 0.0983 - val_auc: 0.5013
Epoch 3/10
35/35 [==============================] - 14s 401ms/step - loss: 0.0919 - auc: 0.5969 - val_loss: 0.1425 - val_auc: 0.5545
Epoch 4/10
35/35 [==============================] - 14s 400ms/step - loss: 0.0829 - auc: 0.7171 - val_loss: 0.0930 - val_auc: 0.7342
Epoch 5/10
35/35 [==============================] - 14s 394ms/step - loss: 0.0755 - auc: 0.7990 - val_loss: 0.1066 - val_auc: 0.7250
Epoch 6/10
35/35 [==============================] - 14s 392ms/step - loss: 0.0711 - auc: 0.8283 - val_loss: 0.0963 - val_auc: 0.7015
Epoch 7/10
35/35 [==============================] - 14s 403ms/step - loss: 0.0665 - auc: 0.8632 - val_loss: 0.0874 - val_auc: 0.7383
Epoch 8/10
35/35 [==============================] - 14s 392ms/step - 

In [11]:
model.load_weights(filepath)
model.save("model.h5")

In [12]:
model.evaluate(val_ds)

171/171 [==============================] - 37s 216ms/step - loss: 0.0809 - auc: 0.7595


[0.0809059664607048, 0.7595189809799194]

In [13]:
test = pd.read_csv(f'{PATH}/test.csv')

test.head()

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge
0,ISIC_0052060,IP_3579794,male,70.0,NaN
1,ISIC_0052349,IP_7782715,male,40.0,lower extremity
2,ISIC_0058510,IP_7960270,female,55.0,torso
3,ISIC_0073313,IP_6375035,female,50.0,torso
4,ISIC_0073502,IP_0589375,female,45.0,lower extremity


In [14]:
def decode_test(filename):
    img = tf.io.read_file(filename)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.cast(img, tf.float32)
    return img

def load_test_ds(df):
    imgs = df["image_name"].values
    imgs = [f'{PATH}/jpeg224/test/{img}.jpg' for img in imgs]
    ds = tf.data.Dataset.from_tensor_slices(imgs)
    ds = ds.map(decode_test)
    ds = ds.batch(BATCH_SIZE)
    return ds

In [15]:
test_ds = load_test_ds(test)

In [16]:
preds = model.predict(test_ds)

In [17]:
preds

array([[0.01247317],
       [0.01131347],
       [0.02000073],
       ...,
       [0.03002178],
       [0.00500779],
       [0.06655625]], dtype=float32)

In [18]:
submission = pd.DataFrame({'image_name': test['image_name'].values, 'target': preds.ravel()})

submission

,image_name,target
0,ISIC_0052060,0.012473
1,ISIC_0052349,0.011313
2,ISIC_0058510,0.020001
3,ISIC_0073313,0.000767
4,ISIC_0073502,0.019851
...,...,...
10977,ISIC_9992485,0.006312
10978,ISIC_9996992,0.046763
10979,ISIC_9997917,0.030022
10980,ISIC_9998234,0.005008


In [19]:
submission.to_csv('submission.csv', index=False)